In [ ]:
import glob
import os
import random

image_path='/home/jason/data/coco/images/'
mode='train2014/'
image_list=[]
image_list.extend(glob.glob(os.path.join(image_path,mode, '*.jpg')))
image_list.sort()
print(len(image_list))

text_path='/home/jason/data/coco/text/'
label_list = []
label_list.extend(glob.glob(os.path.join(text_path,mode, '*.txt')))
label_list.sort()
print(len(label_list))

with open(label_list[0], "r") as f:
    data = f.readlines()
    label = random.choice(data)
    print(label)

In [9]:
import os
import torch
import glob
from PIL import Image
import random
import clip

device = "cuda:3" if torch.cuda.is_available() else "cpu"
_, preprocess = clip.load('ViT-B/32', device)

class cocodtrain(torch.utils.data.Dataset):
    def __init__(self, image_path='/home/jason/data/coco/images', text_path='/home/jason/data/coco/text', mode='train2014', input_resolution=224):

        self.image_list = []
        self.image_list.extend(glob.glob(os.path.join(image_path, mode, '*.jpg')))
        self.image_list.sort()

        self.label_list = []
        self.label_list.extend(glob.glob(os.path.join(text_path, mode, '*.txt')))
        self.label_list.sort()

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, index):
        image = Image.open(self.image_list[index])
        image = preprocess(image)

        # image.Compose([
        #         Resize(224, interpolation=BICUBIC),
        #         CenterCrop(224),
        #         _convert_image_to_rgb,
        #         ToTensor(),
        #         Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
        #                 ])

        with open(self.label_list[index], "r") as f:
            data = f.readlines()
            label = random.choice(data)
            label = clip.tokenize(label)
            #print(label)


        return image.to(device), label.to(device)


In [ ]:
from clip.model import Gauss_model
model_g = Gauss_model().to(device)

In [11]:

from tqdm.notebook import tqdm

trainset = cocodtrain('/home/jason/data/coco/images','/home/jason/data/coco/text','train2014',224)
trainloader = torch.utils.data.DataLoader(
                    trainset, 
                    batch_size=32,
                    shuffle=True, 
                    num_workers=2)

for epoch in range(2): 
        for i, data in enumerate(tqdm(trainloader, 0)):
                images, text = data
                image_features,text_features = model_g(images,text)
                
      

  0%|          | 0/2587 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [19]:
# Latest Update : 22 June 2021, 09:55 GMT+7

# TO ADD :
# Gradient Checkpointing
# Filter out bias from weight decay
# Decaying learning rate with cosine schedule
# Half-precision Adam statistics
# Half-precision stochastically rounded text encoder weights were used

#BATCH_SIZE must larger than 1
train_dataloader = DataLoader(...,batch_size = BATCH_SIZE) #Define your own dataloader

#https://github.com/openai/CLIP/issues/57
def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float() 

device = "cuda:0" if torch.cuda.is_available() else "cpu" # If using GPU then use mixed precision training.
model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training
if device == "cpu":
  model.float()
else :
  clip.model.convert_weights(model) # Actually this line is unnecessary since clip by default already on float16

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2) #Params used from paper, the lr is smaller, more safe for fine tuning to new dataset

for epoch in range(EPOCH):
  for batch in train_dataloader :
      optimizer.zero_grad()

      list_image,list_txt = batch #list_images is list of image in numpy array(np.uint8), or list of PIL images
    
      images= torch.stack([preprocess(Image.fromarray(img)) for img in list_image],dim=0).to(device) # omit the Image.fromarray if the images already in PIL format, change this line to images=list_image if using preprocess inside the dataset class
      texts = clip.tokenize(list_txt).to(device)
    
      logits_per_image, logits_per_text = model(images, texts)

      ground_truth = torch.arange(BATCH_SIZE,dtype=torch.long,device=device)

      total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
      total_loss.backward()
      if device == "cpu":
         optimizer.step()
      else : 
        convert_models_to_fp32(model)
        optimizer.step()
        clip.model.convert_weights(model)

NameError: name 'DataLoader' is not defined

In [ ]:
torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': total_loss,
        }, f"model_checkpoint/model_10.pt") #just change to your preferred folder/filename
        
model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training
checkpoint = torch.load("model_checkpoint/model_10.pt")

# Use these 3 lines if you use default model setting(not training setting) of the clip. For example, if you set context_length to 100 since your string is very long during training, then assign 100 to checkpoint['model_state_dict']["context_length"] 
checkpoint['model_state_dict']["input_resolution"] = model.input_resolution #default is 224
checkpoint['model_state_dict']["context_length"] = model.context_length # default is 77
checkpoint['model_state_dict']["vocab_size"] = model.vocab_size 

model.load_state_dict(checkpoint['model_state_dict'])